#### Branch and Bound
João Antonio Honorato, matrícula: 20210026680

Maria Raquel Martinez, matrícula: 20200025900

In [37]:
from mip import (CBC, CONTINUOUS, MAXIMIZE, Model, xsum)

In [38]:
## A função cria_problema lê o arquivo txt com os dados de entrada
## e os traduz para um modelo do tipo Model
## recebe duas strings, o caminho para o arquivo e um nome para o problema (opcional)
def cria_problema(caminho, nome = ""):

        ## lendo o arquivo:

    entrada = open(caminho, "r") 

    linhas = entrada.readlines()
    # lista de strings, cada elemento é uma linha da entrada

    n_var = int(linhas[0].split()[0]) # número de variáveis
    n_restr = int(linhas[0].split()[1]) # número de restrições

        ## instanciando o problema:

    modelo = Model(nome, sense = MAXIMIZE, solver_name = CBC)

    modelo.verbose = 0

    x = {i: modelo.add_var(var_type = CONTINUOUS, name = f'x{i+1}', lb = 0.0, ub = 1.0,) for i in range(n_var)}

    funcao_objetivo = [int(a) for a in linhas[1].split()]

    modelo.objective = xsum(funcao_objetivo[i] * x[i] for i in range(n_var))

    for i in range(n_restr):
        coeficientes = [int(a) for a in linhas[i+2].split()]
        modelo += xsum(coeficientes[j] * x[j] for j in range(n_var)) <= coeficientes[-1]
    
    status = modelo.optimize()

    entrada.close()

    return modelo

In [39]:
## imprime as informações da solução
def imprime_solucao(modelo):
  print("Problema: " + modelo.name)
  print("Status: ", modelo.status)
  print(f"Valor da solução: {modelo.objective_value:.2f}")
  
  print("Variáveis:")
  for v in modelo.vars:
      print(f"  {v.name} = {v.x:.2f}")

In [40]:
## imprime nome, objetivo, função objetivo e restrições
def imprime_modelo(model):
  model.write("model.lp") # salva modelo em arquivo
  with open("model.lp", "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())
  
  # fazemos assim porque write e open são métodos próprios da classe Model
  # porém não tem um método para imprimí-lo sem essa gambiarra

In [93]:
def ramifica(modelo : Model):

    ## escolhendo a variável a ser "restringida"
    ## aquela cujo valor na solução a ser ramificada mais se aproxima de 0.5

    i = 0 # contador, acompanha as variáveis percorridas
    menor_diferenca = float("inf")
    variavel_indice = i # índice da variável que escolheremos
    for v in modelo.vars: 
        diferenca = abs(v.x - 0.5)
        if menor_diferenca > diferenca:
            menor_diferenca = diferenca
            variavel_indice = i
        i+=1

    ## criando dois novos modelos, iguais ao anterior
    ## cada um com uma restrição nova, xi = 0 e xi = 1  

    modelo_novo_0 = modelo.copy()
    modelo_novo_1 = modelo.copy()
    modelo_novo_0 += modelo_novo_0.vars[variavel_indice] == 0
    modelo_novo_1 += modelo_novo_1.vars[variavel_indice] == 1

    return modelo_novo_0, modelo_novo_1

In [96]:
# TESTE

modelo_teste = cria_problema("testes/teste0.txt", "TESTE 0")
# imprime_solucao(modelo_teste)
a, b = ramifica(modelo_teste)
imprime_modelo(b)

\Problem name: TESTE 0

Minimize
OBJROW: -5 x1 -10 x2 -8 x3
Subject To
constr(0):  3 x1 + 5 x2 + 2 x3 <= 6
constr(1):  4 x1 + 4 x2 + 4 x3 <= 7
constr(2):  x2 = 1
Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
End

